In [41]:
from numpy import zeros, ones, expand_dims, asarray
from numpy.random import randn, randint
from keras.models import Sequential, load_model,Model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from keras import initializers
from keras.initializers import RandomNormal
from matplotlib import pyplot
import numpy as np
from math import sqrt
import pickle

In [3]:
#chargement des données
with open('liste_prot', 'rb') as f:
    X_train = pickle.load(f)
X_train = np.array(X_train)

In [50]:
#test,train
X_train,X_test = train_test_split(X_train,test_size = 0.2)

In [52]:
#700 proteines de 939 aa et 21 aa différents
X_test.shape

(700, 939, 21)

In [45]:
def build_autoencoder(in_shape=(939, 21, 1), size_mini = 10) :
    autoencoder = Sequential()
    #encoder
    autoencoder.add(Input(shape=in_shape))
    autoencoder.add(Flatten())
    autoencoder.add(Dense(512))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dropout(0.3))
    autoencoder.add(Dense(256))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dropout(0.3))
    autoencoder.add(Dense(size_mini))
    #decoder
    autoencoder.add(Dense(256))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dense(400))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dense(512))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dense(939 * 21 * 1))
    autoencoder.add(Reshape((939, 21, 1)))
    autoencoder.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return autoencoder

In [38]:
autoencoder = build_autoencoder()

In [39]:
autoencoder.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 19719)             0         
                                                                 
 dense_40 (Dense)            (None, 512)               10096640  
                                                                 
 leaky_re_lu_29 (LeakyReLU)  (None, 512)               0         
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 dense_41 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_30 (LeakyReLU)  (None, 256)               0         
                                                                 
 dropout_15 (Dropout)        (None, 256)             

In [54]:
#fitting même entrée que la sortie
autoencoder.fit(X_train,X_train,epochs=20, batch_size=128,validation_data = (X_test,X_test))

Epoch 1/20
22/22 [==============================] - 5s 231ms/step - loss: 0.3456 - accuracy: 0.9824 - val_loss: 0.4019 - val_accuracy: 0.9818
Epoch 2/20
22/22 [==============================] - 5s 234ms/step - loss: 0.3355 - accuracy: 0.9829 - val_loss: 0.3991 - val_accuracy: 0.9823
Epoch 3/20
22/22 [==============================] - 5s 234ms/step - loss: 0.3263 - accuracy: 0.9832 - val_loss: 0.3945 - val_accuracy: 0.9822
Epoch 4/20
22/22 [==============================] - 5s 231ms/step - loss: 0.3186 - accuracy: 0.9835 - val_loss: 0.3909 - val_accuracy: 0.9828
Epoch 5/20
22/22 [==============================] - 5s 235ms/step - loss: 0.3112 - accuracy: 0.9839 - val_loss: 0.3890 - val_accuracy: 0.9830
Epoch 6/20
22/22 [==============================] - 5s 234ms/step - loss: 0.3049 - accuracy: 0.9842 - val_loss: 0.3868 - val_accuracy: 0.9830
Epoch 7/20
22/22 [==============================] - 5s 233ms/step - loss: 0.2968 - accuracy: 0.9845 - val_loss: 0.3827 - val_accuracy: 0.9836
Epoch 

In [60]:
#on récupère le decoder qui correspond aux dernières couches
decoder = Sequential()
decoder.add(Input(shape=10))
for layer in autoencoder.layers[8:]: decoder.add(layer)

In [61]:
decoder.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)            (None, 256)               2816      
                                                                 
 leaky_re_lu_31 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_44 (Dense)            (None, 400)               102800    
                                                                 
 leaky_re_lu_32 (LeakyReLU)  (None, 400)               0         
                                                                 
 dense_45 (Dense)            (None, 512)               205312    
                                                                 
 leaky_re_lu_33 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_46 (Dense)            (None, 19719)           

In [63]:
#on créée une nouvelle protéine
test = [[0,1,0,1,0,1,0,1,0,1]]
prot = decoder.predict(test)

In [66]:
prot.shape

(1, 939, 21, 1)

In [ ]:
#to_do :
#fonction prenant des nombres aléatoires pour la création de nouvelles prot
#passer de AE à VAE en rajoutant l'étape de sampling
#fonction permettant de passer du tableau en 939*21 à une séquence type "HWY..."